In [ ]:
from random import randint

N = 8

In [ ]:
class State:
    def __init__(self, board: str) -> None:
        self.board: str = board

    @classmethod
    def random_state(cls) -> 'State':
        res = ''

        for col in range(N):
            res += str(randint(1, N))

        return cls(res)

    @staticmethod
    def heuristic(board: str) -> int:
        """The heuristic cost function h is the number of pairs of queens that are attacking each other;
        this will be zero only for solutions. (It counts as an attack if two pieces are in the same line,
        even if there is an intervening piece between them.)
        """
        h = 0

        for i in range(N):
            # Value in column i
            row_i = int(board[i])

            for j in range(i + 1, N):
                # Value in column j
                row_j = int(board[j])

                # Same row
                if row_i == row_j:
                    h += 1

                # Same diagonal
                if (j - i) == abs(row_j - row_i):
                    h += 1

        return h

    def successors(self) -> list[str]:
        """Generate all the successors of a state, i.e. all possible states generated by moving a single queen
        to another square in the same column (so each state has 8×7=56 successors)."""
        successors = []

        for col in range(N):
            current_row = int(self.board[col])

            for new_row in range(1, N + 1):
                if new_row != current_row:
                    # Create new board with queen moved
                    new_board = self.board[:col] + str(new_row) + s.board[col + 1 :]
                    successors.append(new_board)

        return successors

    def display(self) -> None:
        grid = [['_'] * N for _ in range(N)]

        for col in range(N):
            current_row = int(self.board[col])
            grid[N - current_row][col] = 'X'

        for row in grid:
            print(' '.join(row))

    def __str__(self) -> str:
        return self.board

    def __repr__(self) -> str:
        return f'State(board={self.board!r})'

In [24]:
s = State('43254323')
s.display()
# s.heuristic()
[
    (successor, h)
    for successor in s.successors()
    if (h := State.heuristic(successor)) <= 12
]

_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ X _ _ _ _
X _ _ _ X _ _ _
_ X _ _ _ X _ X
_ _ X _ _ _ X _
_ _ _ _ _ _ _ _


[('46254323', 12),
 ('48254323', 12),
 ('43251323', 12),
 ('43257323', 12),
 ('43254623', 12),
 ('43254823', 12),
 ('43254313', 12),
 ('43254373', 12)]

56

In [6]:
r = State.random_state()
r

State(board='55768132')